In [1]:
import os
import numpy as np
import pandas as pd
from datetime import datetime
import pickle

from aggregate import *
from markup import *

from sklearn.model_selection import train_test_split

In [2]:
data_path = './data'

In [3]:
file = 'ETHBTC_2020-01-01_2020-04-10_1719370.csv'
dataset = 'dataset_0.npz'

In [4]:
### Входные данные

df = pd.read_csv(os.path.join(data_path, file))
df = dtypes_corr(df)
print(f'Количество записей в ленте {df.shape[0]}')
# агрегация данных
frame = 10 # Размер бара. 
# Для метода money_repr и пары ETHBTC означает проторгованный объем в 10 BTC

df = money_repr(df, frame)
df = fill_empty(df)
print(f'Количество баров после агрегации {df.shape[0]}')

# параметры разметки
point = 1e6 # множитель для приведения котировки к целому числу
SL = 100 # StopLoss
TP = 200 # TakeProfit
size_0 = 50 # размер окна в барах, определяющего объект обучения
size_1 = 100 # размер окна в барах, определяющего метку класса

# нарезка на объекты с метками
data = window_cutter(
    df, TP=TP, SL=SL, 
    size_0=size_0, size_1=size_1, 
    point=point
)

Количество записей в ленте 1719370
Количество баров после агрегации 20962


In [5]:
data.shape

(20812, 301)

In [6]:
np.savez_compressed(
    os.path.join(data_path, dataset), 
    obj=data[:,:-1], labels=data[:,-1]
)

Загрузка данных и подготовка к обучению

In [7]:
load_file = os.path.join(data_path, dataset)

In [8]:
X = np.load(load_file)['obj']
y = np.load(load_file)['labels']

Для тренировочных данных рекомендую применить выборку, т.к. объекты идут последовательно с наложением и происходит сильное дублирование обучающей информации, что в свою очередь приводит к переобучению.

In [9]:
# разбиение на трейн тест
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1
)

# создание выборки из тренировочных данных
data = np.hstack([
    X_train, 
    y_train.reshape(y_train.shape[0], 1)
])
sel_idx = select(df, data, size_0)
X_train = X_train[sel_idx]
y_train = y_train[sel_idx]

In [10]:
X_train.shape, y_train.shape

((8257, 300), (8257,))

In [11]:
X_test.shape, y_test.shape

((4163, 300), (4163,))